### Новый поиск-матчинг

In [2]:
from __future__ import annotations

import json
import math
import pickle
import sqlite3
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Iterable, List, Sequence

import numpy as np
import pandas as pd
from whoosh import scoring
from whoosh.analysis import StemmingAnalyzer
from whoosh.fields import ID, TEXT, Schema
from whoosh.filedb.filestore import RamStorage
from whoosh.query import And, FuzzyTerm, Or, Term

In [ ]:
PROJECT_ROOT = Path(".").resolve()
SHOP_DB_PATH = PROJECT_ROOT / "shop.db"
EASE_MODEL_PATH = PROJECT_ROOT / "ease_model.pkl"
POV_MAPPING_PATH = PROJECT_ROOT / "items_rows_pov2vv.json"
VV_CATALOG_PATH = PROJECT_ROOT / "vv_to_pov.csv"

In [102]:
import sqlite3
import pandas as pd

with sqlite3.connect("shop.db") as conn:
    zero_price = pd.read_sql_query(
        "SELECT id, name, price FROM product WHERE price = 0", conn
    )
len_zero = len(zero_price)
len_zero

427

In [103]:
import sqlite3

with sqlite3.connect(SHOP_DB_PATH) as conn:
    total_products = conn.execute("SELECT COUNT(DISTINCT id) FROM product").fetchone()[0]

total_products

8709

In [104]:
len_zero/total_products

0.0490297393500976

5% продуктов имеют цену 0, убирать не очень, поэтому заполним медианой по категории

In [ ]:
def load_product_meta(db_path: Path) -> pd.DataFrame:
    with sqlite3.connect(db_path) as conn:
        df = pd.read_sql_query(
            "SELECT id AS vv_id, name, price, rating, category_id FROM product",
            conn,
        )
    return df

product_meta_df = load_product_meta(SHOP_DB_PATH)

median_by_category = (
    product_meta_df.loc[product_meta_df["price"] > 0]
    .groupby("category_id")["price"]
    .median()
)

def fill_zero_price(row):
    if row["price"] > 0:
        return row["price"]
    return median_by_category.get(row["category_id"], median_by_category.median())

product_meta_df["price"] = product_meta_df.apply(fill_zero_price, axis=1)

In [4]:
@dataclass(frozen=True)
class MatchResult:
    pov_id: int
    vv_id: int
    ingredient: str
    queries: List[str]


In [ ]:
class WhooshCatalogMatcher:
    """Lightweight in-memory Whoosh matcher for arbitrary catalogues."""

    def __init__(self, df: pd.DataFrame, id_col: str = "id", text_col: str = "name"):
        normalized = (
            df[[id_col, text_col]]
            .dropna()
            .drop_duplicates(subset=[id_col])
            .rename(columns={id_col: "id", text_col: "name"})
        )
        normalized["id"] = normalized["id"].astype(int)
        normalized["name"] = normalized["name"].astype(str)
        self.df = normalized.reset_index(drop=True) 
        self.schema = Schema( # схема для индекса (как именно будут храниться данные в индексе)
            id=ID(stored=True, unique=True),
            name=TEXT(stored=True, analyzer=StemmingAnalyzer()),
            name_exact=TEXT(stored=True),
        )
        self._index = self._build_index()

    def _build_index(self): # создаёт индекс для поиска
        storage = RamStorage()
        idx = storage.create_index(self.schema)
        writer = idx.writer()
        for row in self.df.itertuples():
            writer.add_document(id=str(row.id), name=row.name, name_exact=row.name)
        writer.commit()
        return idx 

    def search(self, query: str, limit: int = 5) -> pd.DataFrame: # поиск по запросу (запрос - это название ингредиента)
        query = (query or "").strip()
        if not query:
            return self.df.head(0)

        terms = [token for token in query.lower().split() if token]
        if not terms:
            return self.df.head(0)

        subqueries = []
        # создаём подзапросы для каждого токена (каждого слова в запросе)
        for token in terms:
            if len(token) < 3:
                subqueries.append(
                    Or([Term("name", token), Term("name_exact", token)])
                )
            else:
                subqueries.append(
                    Or(
                        [
                            FuzzyTerm("name", token, maxdist=1, prefixlength=2),
                            Term("name_exact", token),
                        ]
                    )
                )

        whoosh_query = And(subqueries) # это общий запрос по всем
        rows: List[Dict[str, str]] = []
        # создаём список для хранения результатов
        with self._index.searcher(weighting=scoring.BM25F()) as searcher:
            hits = searcher.search(whoosh_query, limit=limit)
            for hit in hits:
                rows.append({"id": int(hit["id"]), "name": hit["name"]})

        if not rows:
            return self.df.head(0)
        return pd.DataFrame(rows)

In [163]:
class EaseOfflineScorer:
    """Minimal EASE scorer that works with the exported pickle artifact."""

    def __init__(self, model_path: Path):
        self.model_path = Path(model_path)
        self.item2idx: Dict[int, int] = {}
        self.idx2item: Dict[int, int] = {}
        self.weights: np.ndarray | None = None
        self._load()

    def _load(self): # загружает модель
        with open(self.model_path, "rb") as f:
            payload = pickle.load(f)

        data = payload if isinstance(payload, dict) else getattr(payload, "__dict__", {})

        value = data.get("model_weights")
        matrix = value
        if hasattr(matrix, "toarray"):
            matrix = matrix.toarray()
        self.weights = np.asarray(matrix) # матрица весов в numpy массив

        value = data.get("item2id")
        mapping = value # маппинг item -> index

        normalized = {int(k): int(v) for k, v in mapping.items()}
        self.item2idx = normalized
        self.idx2item = {int(k): int(v) for k, v in data["id2item"].items()}

    # рекомендует товары, которые похожи на те, что в корзине
    def recommend(self, cart_items: Sequence[int], top_k: int = 50) -> List[tuple[int, float]]:
        if not cart_items or self.weights is None:
            return []

        # Конвертируем pov_id в индексы строк/столбцов. Если каких-то id нет в модели — пропускаем
        seed_idx = [self.item2idx.get(int(item)) for item in cart_items] # получаем индексы строк/столбцов
        seed_idx = [idx for idx in seed_idx if idx is not None] 
        if not seed_idx:
            return []

        # Суммируем веса по строкам, чтобы получить рейтинг для каждого товара
        scores = self.weights[seed_idx].sum(axis=0)
        if scores.ndim > 1:
            scores = scores.sum(axis=0)
        scores = np.asarray(scores, dtype=np.float64)

        # не рекомендуем то, что уже в корзине
        scores[seed_idx] = -np.inf 

        k = min(top_k, scores.size)
        if k <= 0:
            return []

        # берем top k наибольших значений
        top_idx = np.argpartition(-scores, k - 1)[:k]
        top_idx = top_idx[np.argsort(scores[top_idx])[::-1]]
        return [(self.idx2item[idx], float(scores[idx])) for idx in top_idx]



In [ ]:
def _min_max(series: pd.Series) -> pd.Series: # нормализует значения в диапазон [0, 1]
    mn, mx = series.min(), series.max()
    if math.isclose(mx, mn):
        return pd.Series(1.0, index=series.index)
    return (series - mn) / (mx - mn)

def map_llm_outputs_to_matches(
    llm_outputs: Sequence[str],
    per_query_limit: int = 3,
) -> List[MatchResult]:
    """
    Берём сырые строки от LLM (на русском), ищем каждую через Whoosh по каталогу Поварёнка,
    затем через `pov_mapping_df` подтягиваем соответствие pov_id -> vv_id

    - per_query_limit: сколько кандидатов брать из Whoosh на одну строку.
    - На выходе возвращаем список MatchResult, где уже есть pov_id (для EASE),
      vv_id и список исходных запросов
    """
    matches: List[MatchResult] = [] # список для хранения результатов
    for raw_query in llm_outputs:
        if not raw_query:
            continue
        df = pov_searcher.search(raw_query, limit=per_query_limit) # ищем в Whoosh по одному запросу
        if df.empty:
            continue
        # df содержит pov_id и найденные названия ингредиентов.
        # Мержим с mapping, чтобы получить vv_id (который нужно потом отражать на сайте)
        enriched = (
            df.merge(
                pov_mapping_df,
                left_on="id",
                right_on="pov_id",
                how="left",
                suffixes=("", "_map"),
            )
            .dropna(subset=["vv_id"])
            .astype({"vv_id": int})
        )
        for row in enriched.itertuples():
            matches.append(
                MatchResult(
                    pov_id=int(row.pov_id),
                    vv_id=int(row.vv_id),
                    ingredient=row.name, # название из каталога Поварёнка
                    queries=[raw_query], # какая LLM-строка привела к этому попаданию
                )
            )
    return matches 

def aggregate_seed_info(matches: Iterable[MatchResult]) -> pd.DataFrame:
    """
    Сворачиваем список MatchResult в DataFrame по vv_id:
      - собираем уникальные LLM-запросы, которые привели к этому товару;
      - собираем уникальные ингредиенты Поварёнка, которые попали в него
    """
    if not matches:
        return pd.DataFrame(columns=["vv_id", "seed_queries", "seed_ingredients"])

    rows = {}
    for match in matches:
        bucket = rows.setdefault(match.vv_id, {"queries": set(), "ingredients": set()})
        for q in match.queries:
            bucket["queries"].add(q)
        bucket["ingredients"].add(match.ingredient) # добавляем ингредиент в множество для каждого товара

    records = []
    # собираем уникальные LLM-запросы и ингредиенты для каждого товара
    for vv_id, payload in rows.items():
        records.append(
            {
                "vv_id": vv_id,
                "seed_queries": sorted(payload["queries"]),
                "seed_ingredients": sorted(payload["ingredients"]),
            }
        )
    return pd.DataFrame(records)


In [ ]:
def load_vv_catalog(path: Path) -> pd.DataFrame:
    """Берём весь список товаров ВкусВилл из vv_to_pov.csv"""
    df = (
        pd.read_csv(path, usecols=["product_id", "product_title"])
        .dropna()
        .drop_duplicates(subset=["product_id"])
        .rename(columns={"product_id": "vv_id", "product_title": "name"})
    )
    df["vv_id"] = df["vv_id"].astype(int)
    df["name"] = df["name"].astype(str)
    return df

In [ ]:
def load_pov_mapping(path: Path) -> pd.DataFrame: # загружаем mapping между pov_id и vv_id
    with open(path, "r", encoding="utf-8") as f:
        records = json.load(f)
    df = pd.DataFrame(records)
    df = df.dropna(subset=["pov_id", "vv_id", "ingredient"])
    df["pov_id"] = df["pov_id"].astype(int)
    df["vv_id"] = df["vv_id"].astype(int)
    return df

pov_mapping_df = load_pov_mapping(POV_MAPPING_PATH) # загружаем mapping между pov_id и vv_id
vv_catalog_df = load_vv_catalog(VV_CATALOG_PATH) # загружаем весь список товаров ВкусВилл из vv_to_pov.csv

pov_catalog_df = pov_mapping_df[["pov_id", "ingredient"]].rename(
    columns={"pov_id": "id", "ingredient": "name"}
)

# создаём Whoosh-индексы для Поварёнка и ВкусВилл
pov_searcher = WhooshCatalogMatcher(pov_catalog_df)
vv_searcher = WhooshCatalogMatcher(vv_catalog_df, id_col="vv_id", text_col="name")
ease_scorer = EaseOfflineScorer(EASE_MODEL_PATH)

DEFAULT_WEIGHTS = {"ease": 0.6, "rating": 0.25, "price": 0.15}

In [165]:
def pov_to_vv_candidates(pov_ids: Sequence[int], limit_per_pov: int = 10) -> pd.DataFrame:
    """Для каждого pov_id ищем до limit_per_pov товаров ВкусВилл через Whoosh"""
    subset = pov_mapping_df[pov_mapping_df["pov_id"].isin(pov_ids)]
    rows = []
    for row in subset.itertuples():
        hits = vv_searcher.search(row.ingredient, limit=limit_per_pov)
        if hits.empty:
            continue
        for hit in hits.itertuples():
            rows.append(
                {
                    "pov_id": row.pov_id,
                    "vv_id": hit.id,
                    "vv_name": hit.name,
                }
            )
    return pd.DataFrame(rows)

In [159]:
import ast
import pandas as pd

recipes_df = pd.read_csv("recipes_normalized-4.csv", usecols=["ingredients_normalized"])

def parse_ingredients(val):
    parsed = ast.literal_eval(val)
    return parsed

recipes_df["pov_ids"] = recipes_df["ingredients_normalized"].apply(parse_ingredients)
popularity_df = (
    recipes_df.explode("pov_ids")
    .dropna(subset=["pov_ids"])
    .groupby("pov_ids")
    .size()
    .reset_index(name="freq")
    .sort_values("freq", ascending=False)
)

In [120]:
popularity_df

,pov_ids,freq
754,Соль,78094
973,Яйцо куриное,58239
405,Масло растительное,57561
461,Мука пшеничная,50690
673,Сахар-песок,50583
...,...,...
585,Побеги,1
174,Заварной крем,1
432,Молоко кедровое,1
913,Цедра,1


In [160]:
DEFAULT_STOP_WORDS = [
    "Соль",
    "Сахар-песок",
    "Перец черный молотый",
    "Мука пшеничная",
    "Сода",
    "Сода гашеная уксусом",
]

popularity_no_stop = popularity_df[
    ~popularity_df["pov_ids"].isin(DEFAULT_STOP_WORDS)
].reset_index(drop=True)

popularity_no_stop

,pov_ids,freq
0,Яйцо куриное,58239
1,Масло растительное,57561
2,Лук репчатый,42061
3,Масло сливочное,37301
4,Чеснок,31338
...,...,...
968,Побеги,1
969,Заварной крем,1
970,Молоко кедровое,1
971,Цедра,1


In [168]:
with open("items_dict (1).json", "r", encoding="utf-8") as f:
    items_dict = json.load(f)

items_lookup_df = pd.DataFrame(
    [(int(k), v) for k, v in items_dict.items()],
    columns=["pov_id", "ingredient_name"],
)

In [169]:
top_pop_pov_ids = ( # id ингредиентов, которые часто встречаются в рецептах
    popularity_no_stop["pov_ids"]
    .map(lambda name: items_lookup_df.loc[
        items_lookup_df["ingredient_name"] == name, "pov_id"
    ].iloc[0])
    .tolist()
)
top_pop_pov_ids[:5]

[4, 18, 10, 15, 3]

In [166]:
def rerank_ease_candidates(
    llm_outputs: Sequence[str],
    per_query_limit: int = 10,
    ease_top_k: int = 80,
    final_top_k: int = 20,
    weights: Dict[str, float] | None = None,
    ease_context_pov_ids: Sequence[int] | None = None,
) -> pd.DataFrame:
    """
    Реранжируем товары, которые получились из LLM-запросов.
    Берём EASE-рекомендации, дополняем их товарами из популярных ингредиентов,
    и сортируем по релевантности с учетом цены и рейтинга
    """
    weights = weights or DEFAULT_WEIGHTS # веса для реранжирования из EASE

    matches = map_llm_outputs_to_matches(llm_outputs, per_query_limit=per_query_limit) # ищем товары из LLM-запросов через Whoosh
    if not matches:
        return pd.DataFrame()

    seed_info_df = aggregate_seed_info(matches) # агрегируем информацию для каждого товара
    if seed_info_df.empty:
        return pd.DataFrame()

    seed_pov_ids = [match.pov_id for match in matches] # id ингредиентов из LLM-запросов

    ease_scores_df = pd.DataFrame(columns=["pov_id", "ease_score"]) # создаём DataFrame для EASE-рекомендаций
    if ease_context_pov_ids: # если есть ингредиенты в корзине
        ease_raw = ease_scorer.recommend(ease_context_pov_ids, top_k=ease_top_k) # считаем EASE-рекомендации
        ease_scores_df = (
            pd.DataFrame(ease_raw, columns=["pov_id", "ease_score"])
            .groupby("pov_id", as_index=False)["ease_score"]
            .max()
        )

    ease_scores_df = ease_scores_df[ease_scores_df["pov_id"].isin(seed_pov_ids)] # оставляем только ингредиенты из LLM-запросов
    if ease_scores_df.empty:
        ease_scores_df = pd.DataFrame(
            {"pov_id": seed_pov_ids, "ease_score": [0.001] * len(seed_pov_ids)}
        )

    vv_candidates_df = pov_to_vv_candidates(seed_pov_ids, limit_per_pov=20) # ищем товары из популярных ингредиентов через Whoosh
    if vv_candidates_df.empty:
        return pd.DataFrame()

    merged = ( # объединяем товары из популярных ингредиентов, EASE-рекомендации и информацию для каждого товара
        vv_candidates_df.merge(ease_scores_df, on="pov_id", how="left")
        .merge(
            seed_info_df.rename(columns={"vv_id": "seed_vv_id"}),
            left_on="pov_id",
            right_on="seed_vv_id",
            how="left",
        )
        .drop(columns=["seed_vv_id"])
    )

    # base_ingredient — общий ингредиент, по которому будем диверсифицировать выдачу
    # Берём первый seed_ingredient; если его нет (NaN), fallback на строковое pov_id

    merged["ease_score"] = merged["ease_score"].fillna(0.001) # заполняем пропущенные значения EASE-рекомендациями
    merged["base_ingredient"] = merged["seed_ingredients"].str[0].fillna(
        merged["pov_id"].astype(str)
    )

    enriched = merged.merge(product_meta_df, on="vv_id", how="left")
    if enriched.empty:
        return pd.DataFrame()

    # Берем метаданные ВкусВилл (rating/price). Пропуски заполняем медианами
    enriched["rating"] = enriched["rating"].fillna(enriched["rating"].median())
    enriched["price"] = enriched["price"].fillna(enriched["price"].median())

    # Нормализуем признаки: ease_score и price -> min-max, rating -> доля от 5
    enriched["ease_norm"] = _min_max(enriched["ease_score"])
    enriched["rating_norm"] = enriched["rating"].clip(lower=0, upper=5) / 5.0
    enriched["price_norm"] = _min_max(enriched["price"])

    # Итоговый rerank_score = взвешенная сумма нормализованных факторов
    enriched["rerank_score"] = (
        weights["ease"] * enriched["ease_norm"]
        + weights["rating"] * enriched["rating_norm"]
        + weights["price"] * enriched["price_norm"]
    )

    enriched = enriched.sort_values("rerank_score", ascending=False)

    # Группируем по base_ingredient и готовим round-robin, чтобы в выдаче чередовались группы
    grouped = {
        key: list(rows.itertuples(index=False))
        for key, rows in enriched.groupby("base_ingredient")
    }

    round_robin: list[pd.Series] = []
    # Пока есть группы и не достигли лимит, берем по одному товару из каждой группы
    while grouped and len(round_robin) < final_top_k:
        empty_keys = []
        # Берем по одному товару из каждой группы
        for key, items in grouped.items():
            if not items:
                empty_keys.append(key)
                continue
            round_robin.append(items.pop(0))
            if len(round_robin) == final_top_k:
                break
        for key in empty_keys:
            grouped.pop(key, None)

    result_df = pd.DataFrame(round_robin)

    # ---- добивка топ-попами ----
    if len(result_df) < final_top_k:
        missing = final_top_k - len(result_df)
        used_vv_ids = set(result_df.get("vv_id", []))

        filler_rows = []
        for pov_id in top_pop_pov_ids:
            if pov_id in seed_pov_ids:
                continue
            pop_hits = pov_to_vv_candidates([pov_id], limit_per_pov=3)
            if pop_hits.empty:
                continue
            pop_hits = pop_hits[~pop_hits["vv_id"].isin(used_vv_ids)]
            if pop_hits.empty:
                continue

            best_hit = pop_hits.iloc[0]
            meta = product_meta_df.loc[product_meta_df["vv_id"] == best_hit.vv_id]
            if meta.empty:
                continue

            filler_rows.append(
                {
                    "vv_id": best_hit.vv_id,
                    "name": meta["name"].iloc[0],
                    "ease_score": enriched["ease_score"].median(),
                    "rating": meta["rating"].iloc[0],
                    "price": meta["price"].iloc[0],
                    "seed_queries": [["top_pop"]],
                    "seed_ingredients": [[items_lookup_df.loc[items_lookup_df["pov_id"] == pov_id, "ingredient_name"].iloc[0]]],
                    "rerank_score": enriched["rerank_score"].min() - 1.0,
                }
            )
            used_vv_ids.add(best_hit.vv_id)

            if len(filler_rows) == missing:
                break

        if filler_rows:
            filler_df = pd.DataFrame(filler_rows)
            result_df = (
                pd.concat([result_df, filler_df], ignore_index=True)
                .head(final_top_k)
            )

    display_cols = [
        "vv_id",
        "name",
        "ease_score",
        "rating",
        "price",
        "seed_queries",
        "seed_ingredients",
        "rerank_score",
    ]
    return result_df[display_cols].reset_index(drop=True)

In [ ]:
def vv_cart_to_pov_ids(cart_vv_ids: Sequence[int], limit: int = 5) -> List[int]:
    """Берём названия товаров из корзины, ищем их среди ингредиентов Поварёнка"""
    rows = product_meta_df[product_meta_df["vv_id"].isin(cart_vv_ids)]
    pov_ids: List[int] = []
    for row in rows.itertuples():
        hits = pov_searcher.search(row.name, limit=limit)
        if hits.empty:
            continue
        pov_ids.extend(int(hit.id) for hit in hits["id"].tolist())
    return pov_ids

In [167]:
example_queries = ["Морковь", "Зелень", "Помидор"]
current_cart_pov_ids = [10, 8, 55, 63, 13]  # "Лук репчатый", "Майонез", "Огурец", "Фасоль", "Картофель"
reranked_recs = rerank_ease_candidates(
    example_queries,
    final_top_k=20,
    ease_context_pov_ids=current_cart_pov_ids,
)
reranked_recs

,vv_id,name,ease_score,rating,price,seed_queries,seed_ingredients,rerank_score
0,2864,Хумус вяленые помидоры и прованские травы,0.201739,4.9,60.0,NaN,NaN,0.515427
1,10380,"Зелень сушеная ""Розмарин""",0.246089,5.0,42.0,NaN,NaN,0.569974
2,2864,Хумус вяленые помидоры и прованские травы,0.014130,4.9,60.0,NaN,NaN,0.258636
3,4876,"Морковь мини, быстрозам",0.452485,4.8,190.0,NaN,NaN,0.934215
4,10377,"Зелень сушеная ""Базилик""",0.246089,4.9,42.0,NaN,NaN,0.564974
5,4873,Морковь ФрешСекрет по-корейски,0.452485,4.9,176.0,NaN,NaN,0.930537
6,10378,"Зелень сушеная ""Лук""",0.246089,4.9,40.0,NaN,NaN,0.563734
7,4874,Морковь кубиками зам,0.452485,4.9,116.0,NaN,NaN,0.893347
8,10379,"Зелень сушеная ""Петрушка""",0.246089,4.9,38.0,NaN,NaN,0.562495
9,4879,Морковь отварная,0.452485,4.8,124.0,NaN,NaN,0.893306


In [156]:
example_queries = ["Масло сливочное", "Ванильный сахар", "Вода"]
current_cart_pov_ids = [0, 4, 302, 151]  # молоко, яйцо куриное, повидло, дрожжи
reranked_recs = rerank_ease_candidates(
    example_queries,
    final_top_k=20,
    ease_context_pov_ids=current_cart_pov_ids,
)
reranked_recs

,vv_id,name,ease_score,rating,price,seed_queries,seed_ingredients,rerank_score
0,9941,Вода минеральная Borjomi газированная пэт,0.001000,5.0,195.0,NaN,NaN,0.298583
1,4575,"Масло сливочное 82,5% Углече Поле Органик",0.589651,4.9,529.0,NaN,NaN,0.995000
2,9933,Вода кокосовая Foco Organic без сахара,0.006035,4.8,355.0,NaN,NaN,0.342298
3,9933,Вода кокосовая Foco Organic без сахара,0.282265,4.8,355.0,NaN,NaN,0.623854
4,9941,Вода минеральная Borjomi газированная пэт,0.006777,5.0,195.0,NaN,NaN,0.304471
5,7188,Сахар Dr. Bakers Ванильный с натуральной ванилью,0.083485,4.9,68.0,NaN,NaN,0.339095
6,9933,Вода кокосовая Foco Organic без сахара,0.001000,4.8,355.0,NaN,NaN,0.337166
7,9943,Вода минеральная Borjomi газированная с/б,0.001000,4.9,155.0,NaN,NaN,0.281437
8,4580,"Масло сливочное Чабан 82,5%",0.589651,4.9,319.0,NaN,NaN,0.931235
9,9930,Вода кокосовая,0.006035,4.8,230.0,NaN,NaN,0.304342


In [129]:
example_queries = ["Крабовые палочки"]
current_cart_pov_ids = [7, 124, 38, 8]  # Салат, кукуруза, рис
reranked_recs = rerank_ease_candidates(
    example_queries,
    final_top_k=20,
    ease_context_pov_ids=current_cart_pov_ids,
)
reranked_recs

,vv_id,name,ease_score,rating,price,seed_queries,seed_ingredients,rerank_score
0,3665,Крабовые палочки зам,0.145557,4.7,355.0,NaN,NaN,0.985000
1,3667,Крабовые палочки Меридиан Prime Crab,0.145557,4.8,254.0,NaN,NaN,0.919861
2,3671,Крабовые палочки Снежный краб,0.145557,4.9,235.0,NaN,NaN,0.911667
3,3668,Крабовые палочки Меридиан Снежный краб,0.145557,4.9,189.0,NaN,NaN,0.879722
4,3664,Крабовые палочки,0.145557,4.8,188.0,NaN,NaN,0.874028
5,3666,Крабовые палочки Меридиан охлажденные,0.145557,4.9,159.0,NaN,NaN,0.858889
6,3669,Крабовые палочки Русское море имитация охлажде...,0.145557,4.8,149.0,NaN,NaN,0.846944
7,3670,Крабовые палочки Русское море имитация с мясом...,0.145557,4.9,139.0,NaN,NaN,0.845000
8,10108,Яйцо куриное от кур свободного выгула,0.145557,4.9,200.0,[[top_pop]],[[Яйцо куриное]],-0.155000
9,4188,Лук репчатый,0.145557,4.8,48.0,[[top_pop]],[[Лук репчатый]],-0.155000
